# Meshing of Discrete Fracture Networks

This tutorial shows how to create a GridBucket that represents a discrete fracture matrix model. Two meshing strategies are available; they differ in how they treat interseciton lines: 
1. A fully conforming mesh, where the fracture grid that meet along the intersection have common nodes. This is the simplest option for numerical methods, with no hanging nodes. However, it requires joint meshing of all fractures in the network.
2. A partially conforming mesh, where the intersection lines are resolved in all fracture grids, but the grid nodes are not necessarily matching. The grid will thus have hanging nodes, and should be treated by numerical methods that can deal with this. In PorePy, try the virtual element method, and probably TPFA. On the positive side, the fracture planes can be meshed independently, which can be a great benefit for complex geometries.

Importantly, both strategies result in a GridBucket which can be used in further numerical computations.

In [4]:
# Imports needed
import numpy as np
import porepy as pp

Create three fractures with a common intersection point at the origin.

In [5]:
f_1 = pp.Fracture(np.array([[-2, -1, 0], [1, -1, 0], [1, 1, 0], [-2, 1, 0]]).T)
f_2 = pp.Fracture(np.array([[-1, 0, -1], [1, 0, -1], [1, 0, 1], [-1, 0, 1]]).T)
f_3 = pp.Fracture(np.array([[0, -1, -1], [0, 1, -1], [0, 1, 1], [0, -1, 1]]).T)

network = pp.FractureNetwork([f_1, f_2, f_3])

## Fully conforming mesh

First create a fully conforming network. This will mesh the fracture networks simultaneously, and ensure that the nodes on the intersections are coinciding

In [6]:
gb_conforming = pp.meshing.dfn(network, conforming=True, verbose=0)

Found no information on mesh sizes. Returning


Created 3 2-d grids with 359 cells
Created 6 1-d grids with 18 cells
Created 1 0-d grids with 1 cells




## Partially conforming mesh
Next, create a non-conforming grid. In this approach, every fracture plane is meshed independently, and the grids are then merged together with the use of hanging nodes. The merging is somewhat complex, but from the user side, it is simple:

In [7]:
gb_non_conforming = pp.meshing.dfn([f_1, f_2, f_3], conforming=False)
# Also print out some information on the mesh
print(gb_non_conforming)

Mixed dimensional grid. 
Maximum dimension 2
Minimum dimension 1
Size of highest dimensional grid: Cells: 478. Nodes: 381
In lower dimensions: 
3 grids of dimension 1



Note that the number of cells in 2d is different (actually higher, controlled by default mesh size parameters). 

Also, observe that the non-conforming mesh has no 0d objects. Including this should not be too much work, but it has not been required yet, thus not prioritized.

### Visualized meshes

In [12]:
# First export
gb_conforming.assign_node_ordering()
gb_non_conforming.assign_node_ordering()
export_conforming = pp.Exporter(gb_conforming, 'dfn_conrforming')
export_conforming.write_vtk()
export_non_conforming = pp.Exporter(gb_non_conforming, 'dfn_non_conforming')
export_non_conforming.write_vtk()

Some manipulation in paraview then produce the following plots

<img src="fig/dfn_conforming.png" alt="Drawing" style="width: 400px;"/>
Conforming mesh

<img src="fig/dfn_non_conforming.png" alt="Drawing" style="width: 400px;"/>
Non-conforming mesh

## Is there a fully non-conforming approach?
The fully non-conforming approach, in the terminology adapted here, would entail meshes that do not conform to the fracture intersections at all. To create such a mesh is very simple, it is a plane discretization of a 2d surface with no constraints. However, the burden of coupling the equation in each fracture plane is then fully transferred to the numerical method. No such approach is implemented in PorePy - to see how to deal with this, see for instance the work of Berrone, Scialo and coworkers in Turin.